In [2]:
import cv2
import os
import pandas as pd
from csv import writer
import qrcode as qc
import csv
import send2trash
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.image import img_to_array

# Student Register

In [1]:
def register_user():
    video=cv2.VideoCapture(0)
    facedetect=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    firstname = str(input("Enter First Name: ")).title().strip()
    lastname = str(input("Enter Last Name: ")).title().strip()
    enroll = str(input("Enter Enrollment No: ")).upper().strip()
    
    nameID = firstname + lastname
    path = 'D://Dl_Model//StudentData//images//' + nameID + "-" + enroll
    
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
        print("Directory created successfully.")
        
        # Code to add user data to CSV file (class.csv)
        df = pd.read_csv("D://Dl_Model//StudentData//class.csv")
        if df.empty:
            lst = [0, (nameID + "-" + enroll)]
        else:        
            df1 = df.tail(1)
            df3 = df1.iloc[0, 0]
            i = str(df3 + 1)
            lst = [i, (nameID + "-" + enroll)]
            
        with open('D://Dl_Model//StudentData//class.csv', 'a') as f_object:
            writer_object = writer(f_object)
            writer_object.writerow(lst)
            f_object.close()

        count = 0
        while True:
            ret, frame = video.read()
            faces = facedetect.detectMultiScale(frame, 1.3, 5)
            for x, y, w, h in faces:
                count = count + 1
                img_name = 'D://Dl_Model//StudentData//images//' + nameID + '-' + enroll + '//' + str(count) + '.jpg'
                print("Creating Images........." + img_name)
                cv2.imwrite(img_name, frame[y:y+h, x:x+w])
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
            cv2.imshow("Registration", frame)
            cv2.waitKey(1)
            if count > 500:
                break

        video.release()
        cv2.destroyAllWindows()

        # Generate QR code
        try:
            inp = firstname + lastname + "-" + enroll
            features = qc.QRCode(version=2, box_size=20, border=2)
            features.add_data(inp)
            features.make(fit=True)
            generate_image = features.make_image(fill_color="black", back_color="white")
            generate_image.save("D://Dl_Model//StudentData//QR//{}.png".format(inp))
            print("Successfully Created Student QR")
        except Exception as e:
            print("Unable to generate QR: ", str(e))
    else:
        print("Directory already exists. Please enter a different Enrollment No.")
        
#register_user()

# Model Training

In [ ]:
# Load label mapping
label_dict = {}
with open('D://Dl_Model//StudentData//class.csv', mode='r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        label_dict[int(row['classid'])] = row['classname']

# Function to load and preprocess images for training
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
            img = cv2.resize(img, (64, 64))  # Resize images to a fixed size
            img = img / 255.0  # Normalize pixel values to be between 0 and 1
            images.append(img)
    return np.array(images)

def G7model_Train():
    # Load training data
    train_data = []
    labels = []
    for folder_name in os.listdir('D://Dl_Model//StudentData//images'):
        folder_path = os.path.join('D://Dl_Model//StudentData//images', folder_name)
        images = load_images_from_folder(folder_path)
        train_data.extend(images)
        labels.extend([folder_name] * len(images))
    
    train_data = np.array(train_data)
    labels = np.array(labels)
    
    # Convert labels to numerical values
    label_encoder = LabelEncoder()
    labels = label_encoder.fit_transform(labels)
    
    # Data augmentation
    datagen = ImageDataGenerator(rotation_range=20, width_shift_range=0.1, height_shift_range=0.1,
                                 shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(train_data, labels, test_size=0.2, random_state=42)
    
    # Reshape data to fit the model
    X_train = np.reshape(X_train, (X_train.shape[0], 64, 64, 1))
    X_test = np.reshape(X_test, (X_test.shape[0], 64, 64, 1))
    
    # Define the CNN model for face recognition
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1), kernel_regularizer='l2'),
        MaxPooling2D(2, 2),
        
        Conv2D(64, (3, 3), activation='relu', kernel_regularizer='l2'),
        MaxPooling2D(2, 2),
        
        Flatten(),
        Dropout(0.5),
        
        Dense(128, activation='relu', kernel_regularizer='l2'),
        Dropout(0.5),
        
        Dense(len(np.unique(labels)), activation='softmax')
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # Train the model with data augmentation
    model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=20, validation_data=(X_test, y_test))
    
    # Evaluate the model
    model.evaluate(X_test, y_test)
    
    # Evaluate the model
    print("[INFO] Evaluating the model...")
    predictions = model.predict(X_test, batch_size=32)
    print(classification_report(y_test, np.argmax(predictions, axis=1), target_names=label_encoder.classes_))
    
    # Save the model to a file
    model.save('D://Dl_Model//Model//new_face_recognition_model.h5')
    
    print(model.summary())


#G7model_Train()


# Face Prediction

In [5]:
# Load label mapping
label_dict = {}
with open('D://Dl_Model//StudentData//class.csv', mode='r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        label_dict[int(row['classid'])] = row['classname']

def G7_prediction():
    
    # Load the face recognition model
    face_model = load_model('D://Dl_Model//Model//new_face_recognition_model.h5')
    
    # Initialize the face detector
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    
    # Initialize the video capture
    cap = cv2.VideoCapture(0)
    
    while True:
        # Read a frame from the video capture
        ret, frame = cap.read()
    
        # Flip the frame horizontally
        flipped_frame = cv2.flip(frame, 1)
    
        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2GRAY)
    
        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
        # List to store names corresponding to detected faces
        detected_names = []
    
        # Loop over the detected faces
        for (x, y, w, h) in faces:
            face_roi = gray[y:y + h, x:x + w]
            face_roi = cv2.resize(face_roi, (64, 64))  # Resize the face to 64x64 pixels
            face_roi = np.expand_dims(face_roi, axis=-1)  # Add channel dimension
            face_roi = face_roi / 255.0  # Normalize pixel values to be between 0 and 1
    
            # Reshape the image to match the input shape expected by the model
            face_roi = np.reshape(face_roi, (1, 64, 64, 1))
    
            prediction = face_model.predict(face_roi)
            predicted_label = np.argmax(prediction)
            print("Raw Predicted:", prediction) 
    
            # Initialize predicted_name with a default value
            predicted_name = "Unknown"
    
            # Check if predicted_label exists in label_dict
            if predicted_label in label_dict.keys():
                predicted_name = label_dict.get(predicted_label, "Unknown")
                print("Predicted Name:", predicted_name)  # Debug print statement
            else:
                print("Label not found in Student List.")
    
            detected_names.append(predicted_name)  # Add the detected name to the list
    
            # Draw a rectangle around the face
            flipped_frame = cv2.rectangle(flipped_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
        # Display the names corresponding to detected faces
        for i, name in enumerate(detected_names):
            cv2.putText(flipped_frame, str(name), (faces[i][0], faces[i][1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
        # Display the frame
        cv2.imshow("Face Prediction (Press Q to Exit)", flipped_frame)
    
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the video capture and close all windows
    cap.release()
    cv2.destroyAllWindows()

#G7_prediction()


# Mark Attendance

In [6]:
# Function to load and preprocess images for training
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert image to grayscale
            img = cv2.resize(img, (64, 64))  # Resize images to a fixed size
            img = img_to_array(img)
            images.append(img)
    return np.array(images)

# Load label mapping
label_dict = {}
with open('D://Dl_Model//StudentData//class.csv', mode='r') as csvfile:
    csvreader = csv.DictReader(csvfile)
    for row in csvreader:
        label_dict[int(row['classid'])] = row['classname']  # Use 'nameID' instead of 'classname'


train_data = []
labels = []
for folder_name in os.listdir('D://Dl_Model//StudentData//images'):
        folder_path = os.path.join('D://Dl_Model//StudentData//images', folder_name)
        images = load_images_from_folder(folder_path)
        train_data.extend(images)
        labels.extend([folder_name] * len(images))

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)


def mark_attendance():
    # Load or create attendance file
    if os.path.exists('attendancelist.csv'):
        attendance_df = pd.read_csv('attendancelist.csv', index_col='nameID')
    else:
        attendance_df = pd.DataFrame(columns=['nameID'])
        attendance_df.set_index('nameID', inplace=True)

    # Get current date in the required format
    current_date = datetime.now().strftime('%Y-%m-%d')

    # Initialize the face detector, video capture, and other components
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    cap = cv2.VideoCapture(0)
    face_model = load_model('D://Dl_Model//Model//new_face_recognition_model.h5')

    while True:
        ret, frame = cap.read()
        flipped_frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(flipped_frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        decoded_objects = decode(flipped_frame)

        for obj in decoded_objects:
            qr_data = obj.data.decode('utf-8')
            #print("QR: ", qr_data)
        
            predicted_nameID = None  # Initialize predicted_nameID
            match_found = False  # Flag to track if a match is found
        
            for (x, y, w, h) in faces:
                face_roi = gray[y:y + h, x:x + w]
                face_roi = cv2.resize(face_roi, (64, 64))
                face_roi = np.expand_dims(face_roi, axis=-1)
                face_roi = face_roi / 255.0
                face_roi = np.reshape(face_roi, (1, 64, 64, 1))
                prediction = face_model.predict(face_roi)
                predicted_label = np.argmax(prediction)
                
                flipped_frame = cv2.rectangle(flipped_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
                if predicted_label in label_dict.keys():
                    predicted_nameID = label_encoder.classes_[predicted_label]  # Get nameID instead of label number
                    print("Face Name: ", predicted_nameID)
                    print("Qr Name: ",qr_data)
                    
                    # Check if the QR code label matches the face recognition label
                    if qr_data == predicted_nameID:
                        match_found = True  # Set the match_found flag to True
                        # Check if the current_date column exists, if not create it
                        if current_date not in attendance_df.columns:
                            attendance_df[current_date] = ''
        
                        # Mark attendance for the student with matching QR code and face label
                        attendance_df.loc[predicted_nameID, current_date] = "Present"
                        print("QR Code and Face Recognition labels matched.")
                        cv2.putText(flipped_frame, "Attendance Marked", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        break  # Exit the loop if attendance is marked
                    else:
                        print("QR Code and Face Recognition labels do not match.")
        
                else:
                    print("Label not found in Student List.")
        
                
        
            if not match_found:
                cv2.putText(flipped_frame, "Unmatched", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
        
            points = obj.polygon
            if len(points) == 4:
                pts = np.array([(pt.x, pt.y) for pt in points], dtype=int)
                cv2.polylines(flipped_frame, [pts], isClosed=True, color=(0, 255, 0), thickness=2)
        
            qr_code_text = f"QR Code: {qr_data}"
            face_recognition_text = f"Face Recognition: {predicted_nameID}"  # Use nameID instead of label number
            cv2.putText(flipped_frame, qr_code_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(flipped_frame, face_recognition_text, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imshow("Attendance System (Press Q to Exit)", flipped_frame)
        attendance_df.to_csv('attendancelist.csv')
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Creating Chart of attendance

In [7]:
def attendance_chart():
    df = pd.read_csv('D:/Dl_Model/Model/attendancelist.csv')

    df.set_index('nameID', inplace=True)

    # Count the number of present days for each student
    df['Student_Present'] = df.apply(lambda row: row.str.count('Present').sum(), axis=1)

    # Count the total number of days (excluding the 'nameID' column)
    total_days = len(df.columns) - 1

    # Calculate the percentage of attendance for each student
    df['Attendance_Percentage'] = ((df['Student_Present'] / total_days) * 100).round(2)

    # Print the DataFrame with total present days and attendance percentage
    text = "Total Classes: "+str(total_days)
    
    # Get the width of the console or terminal
    terminal_width = os.get_terminal_size().columns

    # Calculate the number of spaces needed to center the text
    left_padding = (terminal_width - len(text)) // 2

    # Printing Total Classes in Center
    print(" " * left_padding + text)
    print("\n")
    print(df[['Student_Present', 'Attendance_Percentage']])

    # Create a pie chart
    for student, attendance_percentage in df['Attendance_Percentage'].items():
        plt.figure(figsize=(6, 6))
        plt.pie([attendance_percentage, 100 - attendance_percentage], labels=['Present', 'Absent'], autopct='%1.1f%%', startangle=140)
        plt.title(f'Attendance Ratio for {student}')
        plt.savefig(f'D:/Dl_Model/StudentData/Attendance_chart/{student}_attendance_chart.png')
        plt.close()

    print("\n Pie charts generated and saved.")


# Showing Charts

In [8]:
def show_chart():
    chart_files = os.listdir('D:/Dl_Model/StudentData/Attendance_chart')

    # Iterate through the chart files and display them
    for chart_file in chart_files:
        if chart_file.endswith('.png'):
            # Create the full file path
            chart_path = os.path.join('D:/Dl_Model/StudentData/Attendance_chart', chart_file)
            
            # Load and display the chart
            img = plt.imread(chart_path)
            plt.figure(figsize=(8, 8))
            plt.imshow(img)
            plt.title(chart_file)  # Set the title as the file name
            plt.axis('off')  # Turn off axis
            plt.show()
        
def show_student_chart():
    firstname = str(input("Enter First Name: ")).title().strip()
    lastname = str(input("Enter Last Name: ")).title().strip()
    enroll = str(input("Enter Enrollment No: ")).upper().strip()
    nameID = firstname + lastname + "-" + enroll
    print(nameID)
    chart_path = os.path.join('D:/Dl_Model/StudentData/Attendance_chart',nameID+'_attendance_chart.png')
            
    # Load and display the chart
    img = plt.imread(chart_path)
    plt.figure(figsize=(8, 8))
    plt.imshow(img)
    plt.axis('off')  # Turn off axis
    plt.show()

# Remove User

In [9]:
def remove_user():
    try:
        firstname = input("Enter First Name: ").strip().title()
        lastname = input("Enter Last Name: ").strip().title()
        enroll = input("Enter Enrollment No: ").strip().upper()
        
        nameID = f"{firstname}{lastname}-{enroll}"

        csv_file_path = r"D:\Dl_Model\StudentData\class.csv"
        image_folder_path = os.path.join(r"D:\Dl_Model\StudentData\images", nameID)
        qr_code_path = os.path.join(r"D:\Dl_Model\StudentData\QR", f"{nameID}.png")

        # Read CSV file
        df = pd.read_csv(csv_file_path)

        # Find user index
        index_to_remove = df.index[df['classname'] == nameID].tolist()

        if index_to_remove:
            # Remove user details from CSV file
            df.drop(index_to_remove, inplace=True)
            df.to_csv(csv_file_path, index=False)
            print("\nUser details removed from CSV.")

            # Delete folder and QR code
            if os.path.exists(image_folder_path):
                try:
                    send2trash.send2trash(image_folder_path)  # Recursively remove directory and its contents
                    print("Image folder deleted.")
                except OSError as e:
                    print(f"Error: {e}")
            else:
                print("Image folder not found.")

            if os.path.exists(qr_code_path):
                os.remove(qr_code_path)
                print("QR code deleted.")
            else:
                print("QR code not found.")
        else:
            print("User not found in the CSV file.")

    except Exception as e:
        print(f"An error occurred: {e}")

#remove_user()


# Home Code


In [6]:
try:
    print("\n           Welcome")
    while True:
        print("-"*40)
        print("Press 1 to register user")
        print("Press 2 to Train Model")
        print("Press 3 for Face Prediction Testing")
        print("Press 4 to Mark Attendance")
        print("Press 5 for Attendace Info and update Graph")
        print("Press 6 for Attendance Chart")
        print("Press 7 to Delete User \n")
        
        choice = input("Enter your choice (-1 for exit the loop): ")
        
        if choice == '1':
            print("Loading...")
            print("Please Wait for a while")
            register_user()
            
        elif choice == '2':
            print("Loading...")
            print("Please Wait for a while")
            G7model_Train()
    
        
        elif choice == '3':
            print("Loading...")
            print("Please Wait for a while")
            G7_prediction()
        
        elif choice == '4':
            print("Loading...")
            print("Please Wait for a while")
            mark_attendance()
            
        elif choice == '5':
            print("Loading...")
            print("Please Wait for a while")
            attendance_chart()
            
        elif choice == '6':
            print("Loading...")
            print("Please Wait for a while")
            show_student_chart()
            
        elif choice == '7':
            print("Loading...")
            print("Please Wait for a while")
            remove_user()
    
        elif choice == '-1':
            break
        
        else:
            print("\n\n","-"*10,"Enter Correct Choice","-"*10,"\n")
            
except ImportError:
    print("Error: Unable to import a module.")
except ValueError:
    print("Error: Invalid input. Please enter a valid number.")
except Exception as e:
    print(f"An error occurred: {e}")
    
finally:
    print("\n","-"*10,"Thankyou for using me. Have a nice day :)","-"*10)


           Welcome
----------------------------------------
Press 1 to register user
Press 2 to Train Model
Press 3 for Face Prediction Testing
Press 4 to Mark Attendance
Press 5 for Attendace Info and update Graph
Press 6 for Attendance Chart
Press 7 to Delete User 

Loading...
Please Wait for a while
An error occurred: name 'G7model_Train' is not defined

 ---------- Thankyou for using me. Have a nice day :) ----------
